In [1]:
# union and union all
import pyspark
import findspark
findspark.init()

In [2]:
# creating a spark session
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("union and union all").getOrCreate()
spark

In [3]:
# Creating a dataframe-1
Data = [("James","Sales","NY",90000,34,10000), 
("Michael","Sales","NY",86000,56,20000), 
("Robert","Sales","CA",81000,30,23000), 
("Maria","Finance","CA",90000,24,23000) 
]
columns= ["employee_name","department","state","salary","age","bonus"]
df1=spark.createDataFrame(data=Data,schema=columns)
df1.show()
df1.printSchema()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
+-------------+----------+-----+------+---+-----+

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)



In [4]:
# Creating a dataframe-2 having same schema as that of df1
Data2 = [("James","Sales","NY",90000,34,10000), 
("Maria","Finance","CA",90000,24,23000), 
("Jen","Finance","NY",79000,53,15000), 
("Jeff","Marketing","CA",80000,25,18000), 
("Kumar","Marketing","NY",91000,50,21000) 
]
columns2= ["employee_name","department","state","salary","age","bonus"]

df2=spark.createDataFrame(Data2,columns2)
df2.show()
df2.printSchema()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)



In [5]:
# 1) UNION
# union() method merges two DataFrames and returns the new DataFrame with all rows from two Dataframes 
#regardless of duplicate data.

df1.union(df2).show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        James|     Sales|   NY| 90000| 34|10000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [6]:
# 2) UNION ALL
# same as union but it is deprecated so recommended to use union

df1.unionall(df2).show()

AttributeError: 'DataFrame' object has no attribute 'unionall'

In [7]:
# 3) Merge without Duplicates

#Since the union() method returns all rows without distinct records, we will use
#the distinct() function to return just one record when a duplicate exists.

df1.union(df2).distinct().show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [8]:
# Handling Null values

# Creating a dataframe containing null values
df1=spark.read.csv("C:\\Users\\DELL\\OneDrive\\Desktop\\Data Engineering Python\\DEPython\\mising.csv",header=True,inferSchema=True)
df1.show()

+------+----+----------+------+
|  Name| age|Experience|Salary|
+------+----+----------+------+
| Krish|  31|        10| 30000|
|  Sona|  30|         8| 25000|
|  Mona|  29|         4| 20000|
| Dhana|  24|         3| 20000|
|  Raju|  21|         1| 15000|
| Bannu|  23|         2| 18000|
|Chinnu|NULL|      NULL| 48999|
|  NULL|  34|        10| 38000|
|  NULL|  36|      NULL|  NULL|
+------+----+----------+------+



In [9]:
# 1. Filter Rows with NULL Values in DataFrame
# use filter() or where()

from pyspark.sql.functions import col

df1.filter(col("Experience").isNull()).show()
df1.filter(df1.Experience.isNull()).show()
df1.filter("Experience is null").show()

+------+----+----------+------+
|  Name| age|Experience|Salary|
+------+----+----------+------+
|Chinnu|NULL|      NULL| 48999|
|  NULL|  36|      NULL|  NULL|
+------+----+----------+------+

+------+----+----------+------+
|  Name| age|Experience|Salary|
+------+----+----------+------+
|Chinnu|NULL|      NULL| 48999|
|  NULL|  36|      NULL|  NULL|
+------+----+----------+------+

+------+----+----------+------+
|  Name| age|Experience|Salary|
+------+----+----------+------+
|Chinnu|NULL|      NULL| 48999|
|  NULL|  36|      NULL|  NULL|
+------+----+----------+------+



In [10]:
# 2. Filter Rows with NULL on Multiple Columns
df1.filter(df1.Name.isNull() & df1.Experience.isNull()).show()

+----+---+----------+------+
|Name|age|Experience|Salary|
+----+---+----------+------+
|NULL| 36|      NULL|  NULL|
+----+---+----------+------+



In [11]:
# 3. Filter Rows with IS NOT NULL or isNotNull
df1.filter(df1.Experience.isNotNull()).show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
|Krish| 31|        10| 30000|
| Sona| 30|         8| 25000|
| Mona| 29|         4| 20000|
|Dhana| 24|         3| 20000|
| Raju| 21|         1| 15000|
|Bannu| 23|         2| 18000|
| NULL| 34|        10| 38000|
+-----+---+----------+------+



In [12]:
# or it can also be written using  na drop
df1.na.drop(subset=["Experience"]).show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
|Krish| 31|        10| 30000|
| Sona| 30|         8| 25000|
| Mona| 29|         4| 20000|
|Dhana| 24|         3| 20000|
| Raju| 21|         1| 15000|
|Bannu| 23|         2| 18000|
| NULL| 34|        10| 38000|
+-----+---+----------+------+



In [13]:
#  fillna() & fill() to replace null values or None values

df1.na.fill(value=10000).show()  # fill null values with 10000 of int type
df1.fillna(value=10000).show()

+------+-----+----------+------+
|  Name|  age|Experience|Salary|
+------+-----+----------+------+
| Krish|   31|        10| 30000|
|  Sona|   30|         8| 25000|
|  Mona|   29|         4| 20000|
| Dhana|   24|         3| 20000|
|  Raju|   21|         1| 15000|
| Bannu|   23|         2| 18000|
|Chinnu|10000|     10000| 48999|
|  NULL|   34|        10| 38000|
|  NULL|   36|     10000| 10000|
+------+-----+----------+------+

+------+-----+----------+------+
|  Name|  age|Experience|Salary|
+------+-----+----------+------+
| Krish|   31|        10| 30000|
|  Sona|   30|         8| 25000|
|  Mona|   29|         4| 20000|
| Dhana|   24|         3| 20000|
|  Raju|   21|         1| 15000|
| Bannu|   23|         2| 18000|
|Chinnu|10000|     10000| 48999|
|  NULL|   34|        10| 38000|
|  NULL|   36|     10000| 10000|
+------+-----+----------+------+



In [14]:
# fill all null values with 0 in age column
df1.na.fill(value=0,subset=["age"]).show()
         # or
df1.fillna(value=0,subset=["age"]).show()

+------+---+----------+------+
|  Name|age|Experience|Salary|
+------+---+----------+------+
| Krish| 31|        10| 30000|
|  Sona| 30|         8| 25000|
|  Mona| 29|         4| 20000|
| Dhana| 24|         3| 20000|
|  Raju| 21|         1| 15000|
| Bannu| 23|         2| 18000|
|Chinnu|  0|      NULL| 48999|
|  NULL| 34|        10| 38000|
|  NULL| 36|      NULL|  NULL|
+------+---+----------+------+

+------+---+----------+------+
|  Name|age|Experience|Salary|
+------+---+----------+------+
| Krish| 31|        10| 30000|
|  Sona| 30|         8| 25000|
|  Mona| 29|         4| 20000|
| Dhana| 24|         3| 20000|
|  Raju| 21|         1| 15000|
| Bannu| 23|         2| 18000|
|Chinnu|  0|      NULL| 48999|
|  NULL| 34|        10| 38000|
|  NULL| 36|      NULL|  NULL|
+------+---+----------+------+



In [15]:
# fill null values with some string
df1.na.fill(value="UNKNOWN").show()

+-------+----+----------+------+
|   Name| age|Experience|Salary|
+-------+----+----------+------+
|  Krish|  31|        10| 30000|
|   Sona|  30|         8| 25000|
|   Mona|  29|         4| 20000|
|  Dhana|  24|         3| 20000|
|   Raju|  21|         1| 15000|
|  Bannu|  23|         2| 18000|
| Chinnu|NULL|      NULL| 48999|
|UNKNOWN|  34|        10| 38000|
|UNKNOWN|  36|      NULL|  NULL|
+-------+----+----------+------+



In [16]:
# Drop Rows with NULL Values in Any Columns
df1.na.drop().show()
df1.dropna().show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
|Krish| 31|        10| 30000|
| Sona| 30|         8| 25000|
| Mona| 29|         4| 20000|
|Dhana| 24|         3| 20000|
| Raju| 21|         1| 15000|
|Bannu| 23|         2| 18000|
+-----+---+----------+------+

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
|Krish| 31|        10| 30000|
| Sona| 30|         8| 25000|
| Mona| 29|         4| 20000|
|Dhana| 24|         3| 20000|
| Raju| 21|         1| 15000|
|Bannu| 23|         2| 18000|
+-----+---+----------+------+



In [17]:
#Drop Rows with NULL Values on All Columns
df1.na.drop(how="all").show()

+------+----+----------+------+
|  Name| age|Experience|Salary|
+------+----+----------+------+
| Krish|  31|        10| 30000|
|  Sona|  30|         8| 25000|
|  Mona|  29|         4| 20000|
| Dhana|  24|         3| 20000|
|  Raju|  21|         1| 15000|
| Bannu|  23|         2| 18000|
|Chinnu|NULL|      NULL| 48999|
|  NULL|  34|        10| 38000|
|  NULL|  36|      NULL|  NULL|
+------+----+----------+------+



In [18]:
#Drop Rows with NULL Values on Selected Columns
df1.na.drop(subset=["Experience"]).show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
|Krish| 31|        10| 30000|
| Sona| 30|         8| 25000|
| Mona| 29|         4| 20000|
|Dhana| 24|         3| 20000|
| Raju| 21|         1| 15000|
|Bannu| 23|         2| 18000|
| NULL| 34|        10| 38000|
+-----+---+----------+------+



In [19]:
# using any
df1.na.drop(how="any").show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
|Krish| 31|        10| 30000|
| Sona| 30|         8| 25000|
| Mona| 29|         4| 20000|
|Dhana| 24|         3| 20000|
| Raju| 21|         1| 15000|
|Bannu| 23|         2| 18000|
+-----+---+----------+------+



In [20]:
# using thresh
#drop rows where there are less than 2 non-null values in a row
# Minimum of two non null values should be present in a row if less than 2 non null values r present it drops off null value.
df1.na.drop(how="any",thresh=2).show()

+------+----+----------+------+
|  Name| age|Experience|Salary|
+------+----+----------+------+
| Krish|  31|        10| 30000|
|  Sona|  30|         8| 25000|
|  Mona|  29|         4| 20000|
| Dhana|  24|         3| 20000|
|  Raju|  21|         1| 15000|
| Bannu|  23|         2| 18000|
|Chinnu|NULL|      NULL| 48999|
|  NULL|  34|        10| 38000|
+------+----+----------+------+



In [21]:
### Date functions
from pyspark.sql.functions import *

data=[["1","2020-02-01"],["2","2019-03-01"],["3","2021-03-01"]]
df=spark.createDataFrame(data,["id","input"])
df.show()

+---+----------+
| id|     input|
+---+----------+
|  1|2020-02-01|
|  2|2019-03-01|
|  3|2021-03-01|
+---+----------+



In [22]:
# 1) current date
df.select(current_date().alias("Today's date")).show(1)

+------------+
|Today's date|
+------------+
|  2024-02-10|
+------------+
only showing top 1 row



In [23]:
# 2) date_format()
df.select(col("input"),date_format(col("input"),"MM-dd-yyyy").alias("Date format changed")).show()

+----------+-------------------+
|     input|Date format changed|
+----------+-------------------+
|2020-02-01|         02-01-2020|
|2019-03-01|         03-01-2019|
|2021-03-01|         03-01-2021|
+----------+-------------------+



In [24]:
# 3) to_date() --- converts string to datatype
df1=df.select(col("input"),to_date(col("input"),"yyyy-MM-dd").alias("to_date")).show()

+----------+----------+
|     input|   to_date|
+----------+----------+
|2020-02-01|2020-02-01|
|2019-03-01|2019-03-01|
|2021-03-01|2021-03-01|
+----------+----------+



In [25]:
# 4) datediff
df.select(col("input"), datediff(current_date(),col("input")).alias("datedifference b/w input and current date")).show()

+----------+-----------------------------------------+
|     input|datedifference b/w input and current date|
+----------+-----------------------------------------+
|2020-02-01|                                     1470|
|2019-03-01|                                     1807|
|2021-03-01|                                     1076|
+----------+-----------------------------------------+



In [26]:
# 5) months_between() ---- months between input and current date
df.select(col("input"), months_between(current_date(),col("input")).alias("months_between")).show()

+----------+--------------+
|     input|months_between|
+----------+--------------+
|2020-02-01|   48.29032258|
|2019-03-01|   59.29032258|
|2021-03-01|   35.29032258|
+----------+--------------+



In [27]:
# 6) add_months() ,  7) date_add(), 8) date_sub()
df.select(col("input"), 
    add_months(col("input"),3).alias("add_months"), 
    add_months(col("input"),-3).alias("sub_months"), 
    date_add(col("input"),4).alias("date_add"), 
    date_sub(col("input"),4).alias("date_sub") 
  ).show()

+----------+----------+----------+----------+----------+
|     input|add_months|sub_months|  date_add|  date_sub|
+----------+----------+----------+----------+----------+
|2020-02-01|2020-05-01|2019-11-01|2020-02-05|2020-01-28|
|2019-03-01|2019-06-01|2018-12-01|2019-03-05|2019-02-25|
|2021-03-01|2021-06-01|2020-12-01|2021-03-05|2021-02-25|
+----------+----------+----------+----------+----------+



In [28]:
# 9) year(), 10) month(), 11) month(), 12) next_day(), 13)weekofyear()

df.select(col("input"), 
     year(col("input")).alias("year"), 
     month(col("input")).alias("month"), 
     next_day(col("input"),"Sunday").alias("next_day"), 
     weekofyear(col("input")).alias("weekofyear") 
  ).show()

+----------+----+-----+----------+----------+
|     input|year|month|  next_day|weekofyear|
+----------+----+-----+----------+----------+
|2020-02-01|2020|    2|2020-02-02|         5|
|2019-03-01|2019|    3|2019-03-03|         9|
|2021-03-01|2021|    3|2021-03-07|         9|
+----------+----+-----+----------+----------+



In [29]:
# 14) dayofweek(), 15) dayofmonth(), 16) dayofyear()

df.select(col("input"),  
     dayofweek(col("input")).alias("dayofweek"), 
     dayofmonth(col("input")).alias("dayofmonth"), 
     dayofyear(col("input")).alias("dayofyear"), 
  ).show()

+----------+---------+----------+---------+
|     input|dayofweek|dayofmonth|dayofyear|
+----------+---------+----------+---------+
|2020-02-01|        7|         1|       32|
|2019-03-01|        6|         1|       60|
|2021-03-01|        2|         1|       60|
+----------+---------+----------+---------+



In [30]:
## Timestamp or Time functions
data=[["1","02-01-2020 11 01 19 06"],["2","03-01-2019 12 01 19 406"],["3","03-01-2021 12 01 19 406"]]
df2=spark.createDataFrame(data,["id","input"])
df2.show(truncate=False)

+---+-----------------------+
|id |input                  |
+---+-----------------------+
|1  |02-01-2020 11 01 19 06 |
|2  |03-01-2019 12 01 19 406|
|3  |03-01-2021 12 01 19 406|
+---+-----------------------+



In [31]:
# 1) current_timestamp()
df2.select(current_timestamp().alias("current_timestamp")
  ).show(1,truncate=False)

+--------------------------+
|current_timestamp         |
+--------------------------+
|2024-02-10 09:30:00.840099|
+--------------------------+
only showing top 1 row



In [32]:
# 2) to_timestamp()
#Converts string timestamp to Timestamp type format.

df2.select(col("input"), to_timestamp(col("input"), "MM-dd-yyyy HH mm ss SSS").alias("to_timestamp")).show(truncate=False)

+-----------------------+-----------------------+
|input                  |to_timestamp           |
+-----------------------+-----------------------+
|02-01-2020 11 01 19 06 |2020-02-01 11:01:19.06 |
|03-01-2019 12 01 19 406|2019-03-01 12:01:19.406|
|03-01-2021 12 01 19 406|2021-03-01 12:01:19.406|
+-----------------------+-----------------------+



In [33]:
# 3) hour(), 4) Minute() and 5) second()
data=[["1","2020-02-01 11:01:19.06"],["2","2019-03-01 12:01:19.406"],["3","2021-03-01 12:01:19.406"]]
df3=spark.createDataFrame(data,["id","input"])


df3.select(col("input"), 
    hour(col("input")).alias("hour"), 
    minute(col("input")).alias("minute"),
    second(col("input")).alias("second") 
  ).show(truncate=False)

+-----------------------+----+------+------+
|input                  |hour|minute|second|
+-----------------------+----+------+------+
|2020-02-01 11:01:19.06 |11  |1     |19    |
|2019-03-01 12:01:19.406|12  |1     |19    |
|2021-03-01 12:01:19.406|12  |1     |19    |
+-----------------------+----+------+------+



In [8]:
# Built in functions
# a) String functions

from pyspark.sql.functions import *

In [14]:
# 1) concat_ws

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

columns = ["name","languagesAtSchool","currentState"]
data = [("James Smith",["Java","Scala","C++"],"CA"), \
    ("Michael Rose",["Spark","Java","C++"],"NJ"), \
    ("Robert Williams",["CSharp","VB"],"NV")]

df = spark.createDataFrame(data=data,schema=columns)
df.printSchema()
df.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- languagesAtSchool: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- currentState: string (nullable = true)

+---------------+------------------+------------+
|name           |languagesAtSchool |currentState|
+---------------+------------------+------------+
|James Smith    |[Java, Scala, C++]|CA          |
|Michael Rose   |[Spark, Java, C++]|NJ          |
|Robert Williams|[CSharp, VB]      |NV          |
+---------------+------------------+------------+



In [15]:


df.withColumn("Student Data",concat_ws("-",df.name,df.languagesAtSchool)).show(truncate=False)

+---------------+------------------+------------+---------------------------+
|name           |languagesAtSchool |currentState|Student Data               |
+---------------+------------------+------------+---------------------------+
|James Smith    |[Java, Scala, C++]|CA          |James Smith-Java-Scala-C++ |
|Michael Rose   |[Spark, Java, C++]|NJ          |Michael Rose-Spark-Java-C++|
|Robert Williams|[CSharp, VB]      |NV          |Robert Williams-CSharp-VB  |
+---------------+------------------+------------+---------------------------+



In [16]:
# 2) length

df.withColumn("Length",length("name")).show()

+---------------+------------------+------------+------+
|           name| languagesAtSchool|currentState|Length|
+---------------+------------------+------------+------+
|    James Smith|[Java, Scala, C++]|          CA|    11|
|   Michael Rose|[Spark, Java, C++]|          NJ|    12|
|Robert Williams|      [CSharp, VB]|          NV|    15|
+---------------+------------------+------------+------+



In [22]:
# 3) lower()
df.select(lower(df.currentState).alias("LOWER CASE")).show()

# 4) upper()
df.select(upper("name").alias("UPPER CASE")).show()

+----------+
|LOWER CASE|
+----------+
|        ca|
|        nj|
|        nv|
+----------+

+---------------+
|     UPPER CASE|
+---------------+
|    JAMES SMITH|
|   MICHAEL ROSE|
|ROBERT WILLIAMS|
+---------------+



In [26]:
# 5) startswith
df.filter(col("name").startswith('R')).show()

+---------------+-----------------+------------+
|           name|languagesAtSchool|currentState|
+---------------+-----------------+------------+
|Robert Williams|     [CSharp, VB]|          NV|
+---------------+-----------------+------------+



In [27]:
# 6) endswith
df.filter(col("currentState").endswith("V")).show()

+---------------+-----------------+------------+
|           name|languagesAtSchool|currentState|
+---------------+-----------------+------------+
|Robert Williams|     [CSharp, VB]|          NV|
+---------------+-----------------+------------+



In [29]:
# 7) substring
df.select(substring("name",5,4)).show()

+---------------------+
|substring(name, 5, 4)|
+---------------------+
|                 s Sm|
|                 ael |
|                 rt W|
+---------------------+



In [32]:
# 8) lpad
df.select(lpad('currentState',5,'0').alias("L-PADDED")).show()  # pad 0 s left side so that length becomes 5

+--------+
|L-PADDED|
+--------+
|   000CA|
|   000NJ|
|   000NV|
+--------+



In [33]:
# 9) rpad
df.select(rpad('currentState',5,'x').alias("R-PADDED")).show()

+--------+
|R-PADDED|
+--------+
|   CAxxx|
|   NJxxx|
|   NVxxx|
+--------+



In [36]:
# 10) trim
df.select(trim(df.name).alias("Final name")).show()

+---------------+
|     Final name|
+---------------+
|    James Smith|
|   Michael Rose|
|Robert Williams|
+---------------+



In [37]:
# 11) ltrim (removes leading spaces)
df.select(ltrim(df.name).alias("Final name")).show()

# 12) rtrim (removes trailing spaces)
df.select(rtrim(df.name).alias("Final name")).show()

+---------------+
|     Final name|
+---------------+
|    James Smith|
|   Michael Rose|
|Robert Williams|
+---------------+

+---------------+
|     Final name|
+---------------+
|    James Smith|
|   Michael Rose|
|Robert Williams|
+---------------+



In [38]:
# 13) split
df.select(split(df.name,' ').alias("After split")).show()

+------------------+
|       After split|
+------------------+
|    [James, Smith]|
|   [Michael, Rose]|
|[Robert, Williams]|
+------------------+



In [41]:
# Math functions

#1) sqrt
df.select(sqrt(lit(16)).alias("Square root")).show(1)

+-----------+
|Square root|
+-----------+
|        4.0|
+-----------+
only showing top 1 row



In [43]:
# 2) abs
df.select(abs(lit(-111)).alias("Absolute")).show(1)

+--------+
|Absolute|
+--------+
|     111|
+--------+
only showing top 1 row



In [44]:
# 3) ceil()
df.select(ceil(lit(12.8)).alias("Ceil")).show(1)

+----+
|Ceil|
+----+
|  13|
+----+
only showing top 1 row



In [45]:
# 4) floor()
df.select(floor(lit(12.8)).alias("Floor")).show(1)

+-----+
|Floor|
+-----+
|   12|
+-----+
only showing top 1 row



In [47]:
# 5) factorial()
df.select(factorial(lit(6)).alias("Factorial")).show(1)

+---------+
|Factorial|
+---------+
|      720|
+---------+
only showing top 1 row



In [49]:
# 6) log10()
df.select(log10(lit(1000)).alias("Log 10 value")).show(1)

+------------+
|Log 10 value|
+------------+
|         3.0|
+------------+
only showing top 1 row



In [51]:
# 7) pow(col1,col2)
df.select(pow(lit(4),lit(2)).alias("Power of a number")).show(1)

+-----------------+
|Power of a number|
+-----------------+
|             16.0|
+-----------------+
only showing top 1 row



In [53]:
# 8) bin(col)
df.select(bin(lit(12)).alias("Binary of number")).show(1)

+----------------+
|Binary of number|
+----------------+
|            1100|
+----------------+
only showing top 1 row



In [54]:
# 9) cbrt(col)
df.select(cbrt(lit(27)).alias("Cube root")).show(1)

+---------+
|Cube root|
+---------+
|      3.0|
+---------+
only showing top 1 row



In [56]:
# when or otherwise


columns = ["name","languagesAtSchool","currentState"]
data = [("James Smith",["Java","Scala","C++"],"CA"), \
    ("Michael Rose",["Spark","Java","C++"],"NJ"), \
    ("Robert Williams",["CSharp","VB"],"NV")]

df.withColumn("New State", when(df.currentState =='CA','Carolina')\
                          .when(df.currentState =='NJ','New Jersey')\
                          .when(df.currentState == 'NZ','New Venezula')\
                          .otherwise(df.currentState)).show()

+---------------+------------------+------------+----------+
|           name| languagesAtSchool|currentState| New State|
+---------------+------------------+------------+----------+
|    James Smith|[Java, Scala, C++]|          CA|  Carolina|
|   Michael Rose|[Spark, Java, C++]|          NJ|New Jersey|
|Robert Williams|      [CSharp, VB]|          NV|        NV|
+---------------+------------------+------------+----------+



In [57]:
# using spark sql
df.createOrReplaceTempView("Person")
spark.sql("select name, CASE WHEN currentState = 'CA' THEN 'Carolina' " + 
               "WHEN currentState = 'NJ' THEN 'New Jersey' WHEN currentState IS NULL THEN ''" +
              "ELSE currentState END as new_State from Person").show()

+---------------+----------+
|           name| new_State|
+---------------+----------+
|    James Smith|  Carolina|
|   Michael Rose|New Jersey|
|Robert Williams|        NV|
+---------------+----------+



In [34]:
# Working with JSON files

#Read JSON file into DataFrame

# 1) using read.json
df=spark.read.json("C:\\Users\\DELL\\Downloads\\zipcodes.json")
df

DataFrame[City: string, Country: string, Decommisioned: boolean, EstimatedPopulation: bigint, Lat: double, Location: string, LocationText: string, LocationType: string, Long: double, Notes: string, RecordNumber: bigint, State: string, TaxReturnsFiled: bigint, TotalWages: bigint, WorldRegion: string, Xaxis: double, Yaxis: double, Zaxis: double, ZipCodeType: string, Zipcode: bigint]

In [35]:
df.show()

+-------------------+-------+-------------+-------------------+-----+--------------------+--------------------+--------------+-------+-------------+------------+-----+---------------+----------+-----------+-----+-----+-----+-----------+-------+
|               City|Country|Decommisioned|EstimatedPopulation|  Lat|            Location|        LocationText|  LocationType|   Long|        Notes|RecordNumber|State|TaxReturnsFiled|TotalWages|WorldRegion|Xaxis|Yaxis|Zaxis|ZipCodeType|Zipcode|
+-------------------+-------+-------------+-------------------+-----+--------------------+--------------------+--------------+-------+-------------+------------+-----+---------------+----------+-----------+-----+-----+-----+-----------+-------+
|        PARC PARQUE|     US|        false|               NULL|17.96|NA-US-PR-PARC PARQUE|     Parc Parque, PR|NOT ACCEPTABLE| -66.22|         NULL|           1|   PR|           NULL|      NULL|         NA| 0.38|-0.87|  0.3|   STANDARD|    704|
|PASEO COSTA DEL SUR

In [40]:
# 2) using format

df=spark.read.format("json").load("C:\\Users\\DELL\\Downloads\\zipcodes.json")
df

DataFrame[City: string, Country: string, Decommisioned: boolean, EstimatedPopulation: bigint, Lat: double, Location: string, LocationText: string, LocationType: string, Long: double, Notes: string, RecordNumber: bigint, State: string, TaxReturnsFiled: bigint, TotalWages: bigint, WorldRegion: string, Xaxis: double, Yaxis: double, Zaxis: double, ZipCodeType: string, Zipcode: bigint]

In [41]:
df.show()

+-------------------+-------+-------------+-------------------+-----+--------------------+--------------------+--------------+-------+-------------+------------+-----+---------------+----------+-----------+-----+-----+-----+-----------+-------+
|               City|Country|Decommisioned|EstimatedPopulation|  Lat|            Location|        LocationText|  LocationType|   Long|        Notes|RecordNumber|State|TaxReturnsFiled|TotalWages|WorldRegion|Xaxis|Yaxis|Zaxis|ZipCodeType|Zipcode|
+-------------------+-------+-------------+-------------------+-----+--------------------+--------------------+--------------+-------+-------------+------------+-----+---------------+----------+-----------+-----+-----+-----+-----------+-------+
|        PARC PARQUE|     US|        false|               NULL|17.96|NA-US-PR-PARC PARQUE|     Parc Parque, PR|NOT ACCEPTABLE| -66.22|         NULL|           1|   PR|           NULL|      NULL|         NA| 0.38|-0.87|  0.3|   STANDARD|    704|
|PASEO COSTA DEL SUR

In [44]:
# 3) Read JSON file from multiline
# Using read.option("multiline","true")

#use multiline option to read JSON files scattered across multiple lines. By default multiline option, is set to false.


df=spark.read.option("multiline","true").json("C:\\Users\\DELL\\Downloads\\zipcodes.json")
df.show(truncate=False)

+-----------+-------+-------------+-----+--------------------+---------------+--------------+------+------------+-----+-----------+-----+-----+-----+-----------+-------+
|City       |Country|Decommisioned|Lat  |Location            |LocationText   |LocationType  |Long  |RecordNumber|State|WorldRegion|Xaxis|Yaxis|Zaxis|ZipCodeType|Zipcode|
+-----------+-------+-------------+-----+--------------------+---------------+--------------+------+------------+-----+-----------+-----+-----+-----+-----------+-------+
|PARC PARQUE|US     |false        |17.96|NA-US-PR-PARC PARQUE|Parc Parque, PR|NOT ACCEPTABLE|-66.22|1           |PR   |NA         |0.38 |-0.87|0.3  |STANDARD   |704    |
+-----------+-------+-------------+-----+--------------------+---------------+--------------+------+------------+-----+-----------+-----+-----+-----+-----------+-------+



In [48]:
# 4) Reading multiple files at a time

df=spark.read.json(["C:\\Users\\DELL\\OneDrive\\Desktop\\Data Engineering Python\\DEPython\\jsondata.json","C:\\Users\\DELL\\OneDrive\\Desktop\\Data Engineering Python\\DEPython\\data1.json"])
df.show(truncate=False)

+-----------------------------------------------------------------------------------------+----------------------+-----+
|_corrupt_record                                                                          |email                 |name |
+-----------------------------------------------------------------------------------------+----------------------+-----+
|{                                                                                        |NULL                  |NULL |
|   "ID":["1","2","3","4","5","6","7","8" ],                                              |NULL                  |NULL |
|   "Name":["Rick","Dan","Michelle","Ryan","Gary","Nina","Simon","Guru" ],                |NULL                  |NULL |
|   "Salary":["623.3","515.2","611","729","843.25","578","632.8","722.5" ],               |NULL                  |NULL |
|   "StartDate":[ "1/1/2012","9/23/2013","11/15/2014","5/11/2014","3/27/2015","5/21/2013",|NULL                  |NULL |
|      "7/30/2013","6/17/2014"],

In [50]:
# 5) Reading files with a user-specified custom schema

# Define custom schema

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, BooleanType
schema = StructType([
      StructField("RecordNumber",IntegerType(),True),
      StructField("Zipcode",IntegerType(),True),
      StructField("ZipCodeType",StringType(),True),
      StructField("City",StringType(),True),
      StructField("State",StringType(),True),
      StructField("LocationType",StringType(),True),
      StructField("Lat",DoubleType(),True),
      StructField("Long",DoubleType(),True),
      StructField("Xaxis",IntegerType(),True),
      StructField("Yaxis",DoubleType(),True),
      StructField("Zaxis",DoubleType(),True),
      StructField("WorldRegion",StringType(),True),
      StructField("Country",StringType(),True),
      StructField("LocationText",StringType(),True),
      StructField("Location",StringType(),True),
      StructField("Decommisioned",BooleanType(),True),
      StructField("TaxReturnsFiled",StringType(),True),
      StructField("EstimatedPopulation",IntegerType(),True),
      StructField("TotalWages",IntegerType(),True),
      StructField("Notes",StringType(),True)
  ])

df_with_schema = spark.read.schema(schema) \
        .json("C:\\Users\\DELL\\Downloads\\zipcodes.json")
df_with_schema.printSchema()
df_with_schema.show()

root
 |-- RecordNumber: integer (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- ZipCodeType: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- LocationType: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Xaxis: integer (nullable = true)
 |-- Yaxis: double (nullable = true)
 |-- Zaxis: double (nullable = true)
 |-- WorldRegion: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- LocationText: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Decommisioned: boolean (nullable = true)
 |-- TaxReturnsFiled: string (nullable = true)
 |-- EstimatedPopulation: integer (nullable = true)
 |-- TotalWages: integer (nullable = true)
 |-- Notes: string (nullable = true)

+------------+-------+-----------+-------------------+-----+--------------+-----+-------+-----+-----+-----+-----------+-------+--------------------+--------------------+------

In [63]:
# Functions in pandas DF
# 1) transform()

#each function takes a pandas Series, and the pandas API on Spark computes the functions in a distributed manner 

# should return same length as output
import pandas as pd
psdf = pd.DataFrame({'a': [1,2,3], 'b':[4,5,6]})
print(psdf)

# Adding 10 to each element

def pandas_plus(count):
     return count + 10  # should always return the same length as input.
    
psdf.transform(pandas_plus)

   a  b
0  1  4
1  2  5
2  3  6


,a,b
0,11,14
1,12,15
2,13,16


In [67]:
# 2) apply


#each function takes a pandas Series, and the pandas API on Spark computes the functions in a distributed manner 

# It is not mandatory to return same length as output

psdf = pd.DataFrame({'a': [1,2,3], 'b':[5,6,7]})
def pandas_plus(x):
    return x[x % 2 == 1]  # allows an arbitrary length

psdf.apply(pandas_plus)

,a,b
0,1,5
2,3,7


In [77]:
import pyspark.pandas as pd
import numpy as np


technologies = ({
    'Fee' :[20000,25000,30000,22000,np.NaN],
    'Discount':[1000,2500,1500,1200,3000]
               })

psdf = pd.DataFrame(technologies)
print(type(psdf))
print(psdf)


def add(data):
   return data[0]+data[1]
  
addDF = psdf.apply(add)
print(addDF)

<class 'pyspark.pandas.frame.DataFrame'>
       Fee  Discount
0  20000.0      1000
1  25000.0      2500
2  30000.0      1500
3  22000.0      1200
4      NaN      3000


C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\pyspark\pandas\utils.py:1016: PandasAPIOnSparkAdviceWarning: If the type hints is not specified for `apply`, it is expensive to infer the data type internally.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


Fee         45000.0
Discount     3500.0
dtype: float64
